In [1]:
# Note: This script will fail to run until the consultant_df is filled with valid entries.


import requests as reqs, pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

def print_summary_report(consultant_name = 'Ryan', fromToList =[]):
    '''
    Prints a CLI summary report for each consultant (last 30 days)
    '''

    def consultant_dataframe():
        '''
        Setting up pandas DF to store consultant identifiers
        '''
        consultant_df = pd.DataFrame( # creating dataframe to hold employee info
            data= {
                'API Key': [
                '#########################'
                ],
                'ID': [
                '#########################'
                ]
             },
            index= [
                'Ryan',
    ]
        )
        return consultant_df

    consultant_df = consultant_dataframe() # Initialize the consultant DF
    #date_range = handle_dates()
        
    API, ID  = consultant_df.loc[consultant_name,'API Key'] , consultant_df.loc[consultant_name,'ID']
    
    
    r = reqs.post( # creating the request object
        url= f'######## PATH ############', #static URL for the summary endpoint
        headers = { # headers for summary request
            "content-type": "application/json", 
            "X-Api-Key":API
        }, 
        json = { # JSON payload to specify additional parameters
            "dateRangeStart": str(fromToList[0]),
            "dateRangeEnd": str(fromToList[1]),
            "summaryFilter": {"groups": ["PROJECT"]},
            "sortOrder": "DESCENDING"
        }
    )
    
    json2dict = dict(r.json())
    df2 = pd.DataFrame(json2dict['groupOne'])
    df2['duration'] /=3600
    df2['Consultant_Name'] = consultant_name
    df2.drop(['amounts','amount','name','nameLowerCase','color'], axis=1, inplace=True)
    
    
    rates_df = pd.read_excel(r'C:\Users\mryan\Desktop\Rates.xlsx')
    #rates_df.dropna(inplace=True)
    rates_df.set_index('User', inplace=True)
    
    #df2.join(rates_df, on='Consultant_Name', how='left')
    df3 = df2.merge(rates_df, how = 'left', left_on=['Consultant_Name', 'clientName'], right_on = ['User','Client'])
    #df3.dropna(inplace=True)
    df3.drop(['_id','clientName','Pay_Rate','To','cc'], axis=1, inplace=True)
    df3.dropna(inplace=True)
    for i in df3:
        df3['Bill_Total'] = df3['duration'] * df3['Bill_Rate']
    
    #print('\n' + consultant_name)
    #display(df3)
    
    return df3
    
    
    #print('\n' + consultant_name)
    #print(df2.loc[0:,['clientName', 'duration']].groupby('clientName').sum())
    #print('--------------------------------------------------------------------------------')



In [2]:
def handle_dates(fromYear= 2022, fromMonth= 1, fromDay= 1, months2Add = 1):
    '''
    Expanding the date selection capabilities for clockify POST req.
    '''
    fromdate = datetime(fromYear,fromMonth,fromDay)
    rd = relativedelta(months=months2Add)
    
    return [fromdate.strftime("%Y-%m-%dT%H:%M:%S.%f"), (fromdate+rd).strftime("%Y-%m-%dT%H:%M:%S.%f")]

def report_printer(
        fYear= 2022,
        fMonth= 1,
        fDay= 1, 
        addMonths = 1,
        names = [
            'Ryan Moore',
            ] ):
    
    fromdate = datetime(fYear,fMonth,fDay)
    
    rd = relativedelta(months=addMonths)
    
    
    #print('--------------------------------------------------------------------------------')
    #print('WFI Consulting Hours Summary' +f"\n From: {str(fromdate)}" + f"\n To: {str(fromdate+rd)}")
    #print('--------------------------------------------------------------------------------')
    
    dfList = []
    for i in names:
        report_df_temp = print_summary_report(i, handle_dates(fYear,fMonth,fDay, addMonths))
        dfList.append(report_df_temp)
    
    # dfList = [print_summary_report(i, handle_dates(fYear,fMonth,fDay, addMonths)) for i in names]
    # return dfList

    return dfList
    
totals_df = pd.concat(report_printer(2022,7,1,1))
totals_df.groupby(['Client']).sum()

KeyError: 'Ryan Moore'

In [ ]:
rates_df = pd.read_excel(r'C:\Users\mryan\Desktop\Rates.xlsx')
rates_df.dropna(inplace=True)